create a dataframe for ham
create a dataframe for spam
get only necessary words removing punctuation, with lower case letters, from both spam and ham
use all words to create a dictionary

create a sparse matrix containing all the words
attach each sparse matrix to each  target, and run classification
train test split

#### IMPORTS OF LIBRARY


In [179]:
import pandas as pd
import re
import nltk
import matplotlib.pyplot as plt
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords



In [181]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [183]:
stemmer = PorterStemmer()
words = stopwords.words("english")


In [185]:
len(words)

179

## CREATING A DATAFRAME FOR HAM AND SPAM


In [120]:
data = pd.read_csv('spam.csv')
data.drop(columns=['Unnamed: 2',"Unnamed: 3", "Unnamed: 4"],axis=1, inplace=True)


In [121]:
labels = data['v1']
data = data['v2']

In [122]:
x_train, x_test, y_train, y_test = train_test_split(data, labels , test_size=0.2, random_state=42,)

PIPELINE FOR PREPROCESSION

In [123]:
# sample = "i loved that you loved it understood"

In [124]:
# def tokenize(sentence):
#     return word_tokenize(sentence)


In [125]:
# def stem_words(sentence):
#     stemmed_words = []
#     punctuations = ['!', '.', ',','?', ]
#     for word in sentence:
#         if word not in punctuations:
#             stemmed_word = stemmer.stem(word)
#             stemmed_words.append(stemmed_word)
        
#     return stemmed_words

In [126]:
# tok_sample = tokenize(sample)
# tok_sample

In [127]:
# x_train = x_train.apply(tokenize)
# x_train = x_train.apply(stem_words)

In [128]:
x_train

1978    No I'm in the same boat. Still here at my moms...
3989    (Bank of Granite issues Strong-Buy) EXPLOSIVE ...
3935       They r giving a second chance to rahul dengra.
4078       O i played smash bros  &lt;#&gt;  religiously.
4086    PRIVATE! Your 2003 Account Statement for 07973...
                              ...                        
3772    I came hostel. I m going to sleep. Plz call me...
5191                               Sorry, I'll call later
5226        Prabha..i'm soryda..realy..frm heart i'm sory
5390                           Nt joking seriously i told
860                   In work now. Going have in few min.
Name: v2, Length: 4457, dtype: object

In [129]:
# vocabulary = []
# for message in x_train:
#     for word in message:
#         vocabulary.append(word)
# vocabulary = set(vocabulary)

In [130]:
# vocabulary


In [157]:
url_pattern = url_pattern = r'https?://\S+|www\.\S+'
email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,7}\b'

In [158]:
class processing_text(BaseEstimator, TransformerMixin):
    def __init__(self,remove_punctuation= True, strip_email=True, to_lowercase= True, replace_url= True, replace_number = True, stemming = True):
        self.remove_punctuation = remove_punctuation
        self.strip_email = strip_email
        self.to_lowercase = to_lowercase
        self.replace_number = replace_number
        self.replace_url = replace_url
        self.stemming = stemming
        self.stemmed_data = []
        self.punctuations = ['!', '.', ',','?', ]
    def fit(self , x, y= None,):
        return self
    def transform(self, x, y=None):
        self.stemmed_data = []
        for text in x:
            tokenize_x = word_tokenize(text)
            url_pattern = url_pattern = r'https?://\S+|www\.\S+'
            email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,7}\b'
            stemmed_sentence = []
            for word in tokenize_x:
                if self.remove_punctuation and word in self.punctuations:
                        continue
                
                if self.replace_url and re.match(url_pattern,word):
                    word = "URL"
                         
                if self.replace_number and re.match(r'\+?[0-9][0-9\-]+', word):
                     word = "NUMBER"
                     
                if self.strip_email and re.match(email_pattern,word):
                     continue    
                if self.to_lowercase:
                     word = word.lower()

                if self.stemming:
                    stemmed_word = stemmer.stem(word)
                    stemmed_sentence.append(stemmed_word)
                else:
                     stemmed_sentence.append(word)


            self.stemmed_data.append(stemmed_sentence)

        
        
        return self.stemmed_data



In [205]:
class bag_of_words(BaseEstimator, TransformerMixin):
    def __init__(self, top_words = 2000):
        self.vocab = []
        self.max_vocab = top_words
        self.bag_of_words = []
        self.top_words = top_words

    def fit(self,X,y= None):
        self.vocab = []
        for text in X:
            for word in text:
                self.vocab.append(word)
        unique_words = pd.Series(self.vocab).value_counts()
        frequent_words = unique_words[0:self.top_words]
        self.vocab = frequent_words
        return self
    def transform(self,X,y=None):
        
        for text in X:
            bag = np.zeros(len(self.vocab),dtype=np.float32)
            for ind, word in enumerate(self.vocab):
                if word in text:
                    bag[ind] = 1.0
            self.bag_of_words.append(bag)
        return self.bag_of_words       


In [206]:
pipeline = Pipeline([
 ('pipe',processing_text()),
 ('bow',bag_of_words())
]
)

In [207]:
pipe_data = pipeline.fit_transform(x_train)

In [204]:
pipe_data

[array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)]

TypeError: NDFrame.where() missing 1 required positional argument: 'cond'